In [1]:
!pip install catboost

     |████████████████████████████████| 66.1MB 45kB/s 


In [1]:
#Import Libraries
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier, Pool
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
!unzip Train_hMYJ020.zip
!unzip Test_ND2Q3bm.zip

unzip:  cannot find or open Train_hMYJ020.zip, Train_hMYJ020.zip.zip or Train_hMYJ020.zip.ZIP.
unzip:  cannot find or open Test_ND2Q3bm.zip, Test_ND2Q3bm.zip.zip or Test_ND2Q3bm.zip.ZIP.


In [3]:
#Import and load dataset
df_train = pd.read_csv('train.csv')
train_dict = pd.read_csv('train_data_dict.csv')
df_test = pd.read_csv('test.csv')
samp = pd.read_csv('sample_submission_lfbv3c3.csv')

In [4]:
df_train['Stay'].unique()

array(['0-10', '41-50', '31-40', '11-20', '51-60', '21-30', '71-80',
       'More than 100 Days', '81-90', '61-70', '91-100'], dtype=object)

In [5]:
#Encoding Target / Label
from sklearn.preprocessing import LabelEncoder, StandardScaler

le = LabelEncoder()
df_train['Stay'] = le.fit_transform(df_train['Stay'])

In [6]:
df_train['train_flag'] = 1
df_test['train_flag'] = 0
df_test['Stay'] = None
print(df_train.shape, df_test.shape)

df_data = pd.concat((df_train, df_test))
print(df_data.shape)

(318438, 19) (137057, 19)
(455495, 19)


In [7]:
#Drop Case_id and patient id - too many unique value
df_data.drop(['case_id','patientid','Admission_Deposit'],axis=1,inplace=True)

In [8]:
#Handling the missing values
df_data['Bed Grade'] = np.where(df_data['Bed Grade'].isna(),1.0,df_data['Bed Grade'])
df_data['City_Code_Patient'] = np.where(df_data['City_Code_Patient'].isna(),0.0,df_data['City_Code_Patient'])

In [9]:
#Changing data type from float to int
df_data['Bed Grade'] = df_data['Bed Grade'].astype(int)
df_data['City_Code_Patient'] = df_data['City_Code_Patient'].astype(int)


In [10]:
#Combined Data Analysis
df_data.shape, df_data.columns, df_data.dtypes

((455495, 16),
 Index(['Hospital_code', 'Hospital_type_code', 'City_Code_Hospital',
        'Hospital_region_code', 'Available Extra Rooms in Hospital',
        'Department', 'Ward_Type', 'Ward_Facility_Code', 'Bed Grade',
        'City_Code_Patient', 'Type of Admission', 'Severity of Illness',
        'Visitors with Patient', 'Age', 'Stay', 'train_flag'],
       dtype='object'),
 Hospital_code                         int64
 Hospital_type_code                   object
 City_Code_Hospital                    int64
 Hospital_region_code                 object
 Available Extra Rooms in Hospital     int64
 Department                           object
 Ward_Type                            object
 Ward_Facility_Code                   object
 Bed Grade                             int64
 City_Code_Patient                     int64
 Type of Admission                    object
 Severity of Illness                  object
 Visitors with Patient                 int64
 Age                            

In [11]:
#Listing Categorical Columns
cat=df_data.select_dtypes(['object']).columns
cat

Index(['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness', 'Age',
       'Stay'],
      dtype='object')

In [12]:
#Removig the Target / Label Column -'Stay'
cat=cat[:-1]
cat

Index(['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Age'],
      dtype='object')

In [13]:
df_data = pd.get_dummies(data=df_data,columns=['Hospital_type_code', 'Hospital_region_code', 'Department', 'Ward_Type',
       'Ward_Facility_Code', 'Type of Admission', 'Severity of Illness',
       'Age'])

In [14]:
df_data.shape, df_data.dtypes

((455495, 51),
 Hospital_code                         int64
 City_Code_Hospital                    int64
 Available Extra Rooms in Hospital     int64
 Bed Grade                             int64
 City_Code_Patient                     int64
 Visitors with Patient                 int64
 Stay                                 object
 train_flag                            int64
 Hospital_type_code_a                  uint8
 Hospital_type_code_b                  uint8
 Hospital_type_code_c                  uint8
 Hospital_type_code_d                  uint8
 Hospital_type_code_e                  uint8
 Hospital_type_code_f                  uint8
 Hospital_type_code_g                  uint8
 Hospital_region_code_X                uint8
 Hospital_region_code_Y                uint8
 Hospital_region_code_Z                uint8
 Department_TB & Chest disease         uint8
 Department_anesthesia                 uint8
 Department_gynecology                 uint8
 Department_radiotherapy               u

In [15]:
#Recovering Train and Test Dataset
df_train, df_test = df_data[df_data.train_flag == 1], df_data[df_data.train_flag == 0]
df_train.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['train_flag'], inplace=True, axis=1)
df_test.drop(['Stay'], inplace=True, axis=1)

In [16]:
print(df_train.shape, df_test.shape)

(318438, 50) (137057, 49)


In [17]:
del df_data

In [18]:
df_train['Stay'] = df_train['Stay'].astype(int)
label = df_train['Stay']
df_train.drop(['Stay'],axis=1,inplace=True)

In [19]:
#df_data.drop(['case_id','patientid','Admission_Deposit'],axis=1,inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(df_train, label, test_size = 0.3, random_state = 0)

In [20]:
feature_cols = df_train.columns.tolist()
#feature_cols.remove('Stay')
#feature_cols.remove('Admission_Deposit')
label_col = 'Stay'
print(feature_cols)

['Hospital_code', 'City_Code_Hospital', 'Available Extra Rooms in Hospital', 'Bed Grade', 'City_Code_Patient', 'Visitors with Patient', 'Hospital_type_code_a', 'Hospital_type_code_b', 'Hospital_type_code_c', 'Hospital_type_code_d', 'Hospital_type_code_e', 'Hospital_type_code_f', 'Hospital_type_code_g', 'Hospital_region_code_X', 'Hospital_region_code_Y', 'Hospital_region_code_Z', 'Department_TB & Chest disease', 'Department_anesthesia', 'Department_gynecology', 'Department_radiotherapy', 'Department_surgery', 'Ward_Type_P', 'Ward_Type_Q', 'Ward_Type_R', 'Ward_Type_S', 'Ward_Type_T', 'Ward_Type_U', 'Ward_Facility_Code_A', 'Ward_Facility_Code_B', 'Ward_Facility_Code_C', 'Ward_Facility_Code_D', 'Ward_Facility_Code_E', 'Ward_Facility_Code_F', 'Type of Admission_Emergency', 'Type of Admission_Trauma', 'Type of Admission_Urgent', 'Severity of Illness_Extreme', 'Severity of Illness_Minor', 'Severity of Illness_Moderate', 'Age_0-10', 'Age_11-20', 'Age_21-30', 'Age_31-40', 'Age_41-50', 'Age_51-6

In [21]:
cat_cols = feature_cols
#cat_cols.remove('Admission_Deposit')
cat_cols

['Hospital_code',
 'City_Code_Hospital',
 'Available Extra Rooms in Hospital',
 'Bed Grade',
 'City_Code_Patient',
 'Visitors with Patient',
 'Hospital_type_code_a',
 'Hospital_type_code_b',
 'Hospital_type_code_c',
 'Hospital_type_code_d',
 'Hospital_type_code_e',
 'Hospital_type_code_f',
 'Hospital_type_code_g',
 'Hospital_region_code_X',
 'Hospital_region_code_Y',
 'Hospital_region_code_Z',
 'Department_TB & Chest disease',
 'Department_anesthesia',
 'Department_gynecology',
 'Department_radiotherapy',
 'Department_surgery',
 'Ward_Type_P',
 'Ward_Type_Q',
 'Ward_Type_R',
 'Ward_Type_S',
 'Ward_Type_T',
 'Ward_Type_U',
 'Ward_Facility_Code_A',
 'Ward_Facility_Code_B',
 'Ward_Facility_Code_C',
 'Ward_Facility_Code_D',
 'Ward_Facility_Code_E',
 'Ward_Facility_Code_F',
 'Type of Admission_Emergency',
 'Type of Admission_Trauma',
 'Type of Admission_Urgent',
 'Severity of Illness_Extreme',
 'Severity of Illness_Minor',
 'Severity of Illness_Moderate',
 'Age_0-10',
 'Age_11-20',
 'Age_21

In [22]:
label_col, len(feature_cols), len(cat_cols)

('Stay', 49, 49)

In [26]:
%%time
model = CatBoostClassifier(loss_function='MultiClass')
model.fit(X_train, y_train)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

Learning rate set to 0.10425
0:	learn: 2.2220037	total: 758ms	remaining: 12m 37s
1:	learn: 2.1048716	total: 1.57s	remaining: 13m 5s
2:	learn: 2.0174991	total: 2.74s	remaining: 15m 11s
3:	learn: 1.9515319	total: 3.56s	remaining: 14m 46s
4:	learn: 1.8979421	total: 4.35s	remaining: 14m 25s
5:	learn: 1.8538672	total: 5.12s	remaining: 14m 8s
6:	learn: 1.8170859	total: 5.88s	remaining: 13m 53s
7:	learn: 1.7857322	total: 6.65s	remaining: 13m 44s
8:	learn: 1.7575225	total: 7.44s	remaining: 13m 39s
9:	learn: 1.7336675	total: 8.23s	remaining: 13m 34s
10:	learn: 1.7148624	total: 8.98s	remaining: 13m 27s
11:	learn: 1.6963543	total: 9.76s	remaining: 13m 23s
12:	learn: 1.6819214	total: 10.5s	remaining: 13m 18s
13:	learn: 1.6698712	total: 11.3s	remaining: 13m 17s
14:	learn: 1.6572507	total: 12.1s	remaining: 13m 12s
15:	learn: 1.6465812	total: 12.9s	remaining: 13m 11s
16:	learn: 1.6380503	total: 13.6s	remaining: 13m 8s
17:	learn: 1.6308242	total: 14.4s	remaining: 13m 5s
18:	learn: 1.6240112	total: 15.

153:	learn: 1.5155062	total: 1m 59s	remaining: 10m 57s
154:	learn: 1.5151946	total: 2m	remaining: 10m 56s
155:	learn: 1.5148799	total: 2m 1s	remaining: 10m 55s
156:	learn: 1.5143926	total: 2m 1s	remaining: 10m 54s
157:	learn: 1.5140046	total: 2m 2s	remaining: 10m 53s
158:	learn: 1.5137375	total: 2m 3s	remaining: 10m 51s
159:	learn: 1.5133220	total: 2m 4s	remaining: 10m 51s
160:	learn: 1.5131469	total: 2m 4s	remaining: 10m 50s
161:	learn: 1.5126930	total: 2m 5s	remaining: 10m 49s
162:	learn: 1.5123073	total: 2m 6s	remaining: 10m 48s
163:	learn: 1.5120970	total: 2m 6s	remaining: 10m 47s
164:	learn: 1.5119691	total: 2m 7s	remaining: 10m 46s
165:	learn: 1.5117710	total: 2m 8s	remaining: 10m 45s
166:	learn: 1.5115059	total: 2m 9s	remaining: 10m 44s
167:	learn: 1.5112919	total: 2m 9s	remaining: 10m 43s
168:	learn: 1.5111432	total: 2m 10s	remaining: 10m 42s
169:	learn: 1.5109296	total: 2m 11s	remaining: 10m 41s
170:	learn: 1.5107616	total: 2m 12s	remaining: 10m 40s
171:	learn: 1.5102092	total

305:	learn: 1.4881614	total: 3m 54s	remaining: 8m 51s
306:	learn: 1.4880215	total: 3m 54s	remaining: 8m 50s
307:	learn: 1.4879067	total: 3m 56s	remaining: 8m 50s
308:	learn: 1.4877883	total: 3m 56s	remaining: 8m 49s
309:	learn: 1.4875832	total: 3m 57s	remaining: 8m 49s
310:	learn: 1.4874614	total: 3m 58s	remaining: 8m 48s
311:	learn: 1.4872708	total: 3m 59s	remaining: 8m 48s
312:	learn: 1.4871633	total: 4m	remaining: 8m 48s
313:	learn: 1.4869490	total: 4m 1s	remaining: 8m 47s
314:	learn: 1.4868378	total: 4m 2s	remaining: 8m 46s
315:	learn: 1.4867568	total: 4m 2s	remaining: 8m 45s
316:	learn: 1.4865703	total: 4m 3s	remaining: 8m 45s
317:	learn: 1.4863429	total: 4m 4s	remaining: 8m 44s
318:	learn: 1.4862130	total: 4m 5s	remaining: 8m 44s
319:	learn: 1.4861379	total: 4m 6s	remaining: 8m 44s
320:	learn: 1.4860092	total: 4m 7s	remaining: 8m 44s
321:	learn: 1.4858656	total: 4m 8s	remaining: 8m 44s
322:	learn: 1.4857536	total: 4m 10s	remaining: 8m 44s
323:	learn: 1.4856753	total: 4m 10s	remai

458:	learn: 1.4701307	total: 6m 31s	remaining: 7m 41s
459:	learn: 1.4700267	total: 6m 32s	remaining: 7m 41s
460:	learn: 1.4699447	total: 6m 33s	remaining: 7m 40s
461:	learn: 1.4698931	total: 6m 34s	remaining: 7m 39s
462:	learn: 1.4698169	total: 6m 35s	remaining: 7m 39s
463:	learn: 1.4697273	total: 6m 36s	remaining: 7m 38s
464:	learn: 1.4696164	total: 6m 37s	remaining: 7m 37s
465:	learn: 1.4694662	total: 6m 38s	remaining: 7m 36s
466:	learn: 1.4693889	total: 6m 38s	remaining: 7m 35s
467:	learn: 1.4692904	total: 6m 39s	remaining: 7m 34s
468:	learn: 1.4691166	total: 6m 40s	remaining: 7m 33s
469:	learn: 1.4690492	total: 6m 41s	remaining: 7m 32s
470:	learn: 1.4689110	total: 6m 42s	remaining: 7m 31s
471:	learn: 1.4688384	total: 6m 42s	remaining: 7m 30s
472:	learn: 1.4687874	total: 6m 43s	remaining: 7m 29s
473:	learn: 1.4686969	total: 6m 44s	remaining: 7m 29s
474:	learn: 1.4685587	total: 6m 45s	remaining: 7m 28s
475:	learn: 1.4684238	total: 6m 46s	remaining: 7m 27s
476:	learn: 1.4682568	total:

611:	learn: 1.4536826	total: 8m 34s	remaining: 5m 26s
612:	learn: 1.4536117	total: 8m 35s	remaining: 5m 25s
613:	learn: 1.4534466	total: 8m 36s	remaining: 5m 24s
614:	learn: 1.4533793	total: 8m 37s	remaining: 5m 23s
615:	learn: 1.4532612	total: 8m 38s	remaining: 5m 22s
616:	learn: 1.4530996	total: 8m 38s	remaining: 5m 22s
617:	learn: 1.4530127	total: 8m 39s	remaining: 5m 21s
618:	learn: 1.4529648	total: 8m 40s	remaining: 5m 20s
619:	learn: 1.4529179	total: 8m 41s	remaining: 5m 19s
620:	learn: 1.4528639	total: 8m 42s	remaining: 5m 18s
621:	learn: 1.4527361	total: 8m 43s	remaining: 5m 17s
622:	learn: 1.4526400	total: 8m 43s	remaining: 5m 17s
623:	learn: 1.4525383	total: 8m 44s	remaining: 5m 16s
624:	learn: 1.4523938	total: 8m 45s	remaining: 5m 15s
625:	learn: 1.4522965	total: 8m 46s	remaining: 5m 14s
626:	learn: 1.4521795	total: 8m 47s	remaining: 5m 13s
627:	learn: 1.4520736	total: 8m 48s	remaining: 5m 12s
628:	learn: 1.4519949	total: 8m 49s	remaining: 5m 12s
629:	learn: 1.4518773	total:

764:	learn: 1.4392824	total: 10m 33s	remaining: 3m 14s
765:	learn: 1.4392172	total: 10m 34s	remaining: 3m 13s
766:	learn: 1.4391882	total: 10m 34s	remaining: 3m 12s
767:	learn: 1.4391090	total: 10m 35s	remaining: 3m 12s
768:	learn: 1.4390188	total: 10m 36s	remaining: 3m 11s
769:	learn: 1.4388987	total: 10m 37s	remaining: 3m 10s
770:	learn: 1.4387869	total: 10m 37s	remaining: 3m 9s
771:	learn: 1.4386958	total: 10m 38s	remaining: 3m 8s
772:	learn: 1.4385993	total: 10m 39s	remaining: 3m 7s
773:	learn: 1.4385169	total: 10m 40s	remaining: 3m 6s
774:	learn: 1.4384174	total: 10m 41s	remaining: 3m 6s
775:	learn: 1.4383387	total: 10m 41s	remaining: 3m 5s
776:	learn: 1.4382771	total: 10m 42s	remaining: 3m 4s
777:	learn: 1.4381559	total: 10m 43s	remaining: 3m 3s
778:	learn: 1.4380828	total: 10m 44s	remaining: 3m 2s
779:	learn: 1.4379853	total: 10m 45s	remaining: 3m 1s
780:	learn: 1.4378764	total: 10m 45s	remaining: 3m 1s
781:	learn: 1.4377632	total: 10m 46s	remaining: 3m
782:	learn: 1.4377066	tot

915:	learn: 1.4262703	total: 12m 51s	remaining: 1m 10s
916:	learn: 1.4262066	total: 12m 52s	remaining: 1m 9s
917:	learn: 1.4261228	total: 12m 53s	remaining: 1m 9s
918:	learn: 1.4260402	total: 12m 54s	remaining: 1m 8s
919:	learn: 1.4259495	total: 12m 54s	remaining: 1m 7s
920:	learn: 1.4258684	total: 12m 55s	remaining: 1m 6s
921:	learn: 1.4257756	total: 12m 56s	remaining: 1m 5s
922:	learn: 1.4256544	total: 12m 57s	remaining: 1m 4s
923:	learn: 1.4255109	total: 12m 58s	remaining: 1m 4s
924:	learn: 1.4254060	total: 12m 59s	remaining: 1m 3s
925:	learn: 1.4253323	total: 12m 59s	remaining: 1m 2s
926:	learn: 1.4252601	total: 13m	remaining: 1m 1s
927:	learn: 1.4251750	total: 13m 1s	remaining: 1m
928:	learn: 1.4250850	total: 13m 2s	remaining: 59.8s
929:	learn: 1.4250059	total: 13m 3s	remaining: 59s
930:	learn: 1.4249439	total: 13m 4s	remaining: 58.1s
931:	learn: 1.4248284	total: 13m 4s	remaining: 57.3s
932:	learn: 1.4247711	total: 13m 5s	remaining: 56.4s
933:	learn: 1.4247013	total: 13m 6s	remain

In [27]:
df_test.shape, df_test.columns

((137057, 49),
 Index(['Hospital_code', 'City_Code_Hospital',
        'Available Extra Rooms in Hospital', 'Bed Grade', 'City_Code_Patient',
        'Visitors with Patient', 'Hospital_type_code_a', 'Hospital_type_code_b',
        'Hospital_type_code_c', 'Hospital_type_code_d', 'Hospital_type_code_e',
        'Hospital_type_code_f', 'Hospital_type_code_g',
        'Hospital_region_code_X', 'Hospital_region_code_Y',
        'Hospital_region_code_Z', 'Department_TB & Chest disease',
        'Department_anesthesia', 'Department_gynecology',
        'Department_radiotherapy', 'Department_surgery', 'Ward_Type_P',
        'Ward_Type_Q', 'Ward_Type_R', 'Ward_Type_S', 'Ward_Type_T',
        'Ward_Type_U', 'Ward_Facility_Code_A', 'Ward_Facility_Code_B',
        'Ward_Facility_Code_C', 'Ward_Facility_Code_D', 'Ward_Facility_Code_E',
        'Ward_Facility_Code_F', 'Type of Admission_Emergency',
        'Type of Admission_Trauma', 'Type of Admission_Urgent',
        'Severity of Illness_Extreme', 

In [28]:
preds = model.predict(df_test)
preds

array([[2],
       [5],
       [2],
       ...,
       [2],
       [1],
       [3]])

In [120]:
preds = le.inverse_transform(preds)
preds

array(['21-30', '51-60', '21-30', ..., '21-30', '11-20', '51-60'],
      dtype=object)

In [121]:
test1 = pd.read_csv('test.csv')

In [122]:
submit = pd.DataFrame({'case_id': test1.case_id, 'Stay': preds})
submit.head()



,case_id,Stay
0,318439,21-30
1,318440,51-60
2,318441,21-30
3,318442,21-30
4,318443,51-60


In [123]:
submit.to_csv('Healthcare2Catboost.csv', index=False)